# Experimentos
ASALKSDJLKJ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Dataframes complementarios (Traducción de IDs)              # 1ra Columna   -> Resto de columnas
df_id_banco = pd.read_csv("data/CatBanco.csv")                # ID Banco      -> Nombre del banco
df_id_emisora = pd.read_csv("data/CatEmisora.csv")            # ID Emisora    -> Nombre Emisora, ID Banco, Emisora??, Tipo envío
df_id_respuesta = pd.read_csv("data/CatRespuestaBancos.csv")  # ID Respuesta  -> Descripción
df_id_listacobro = pd.read_csv("data/ListaCobro.csv")         # ID ListaCobro -> Fecha creación, ID Banco, Fecha envío
df_id_listaemisor = pd.read_csv("data/ListaCobroEmisora.csv") # ID ListaCobro -> ID Emisora

# Dataframes detallados por año
df_2022 = pd.read_csv("data/ListaCobroDetalle2022.csv")
df_2023 = pd.read_csv("data/ListaCobroDetalle2023.csv")
df_2024 = pd.read_csv("data/ListaCobroDetalle2024.csv")
df_2025 = pd.read_csv("data/ListaCobroDetalle2025.csv")

